calculate the core power for BOINC tasks

In [6]:
%matplotlib inline
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import time
from elasticsearch import Elasticsearch, helpers

In [7]:
last_days=150
Power=[{'pat':r'.*aws16.*ihep.ac.cn','pvalue':16},{'pat':r'.*lwn14.*ihep.ac.cn','pvalue':20},{'pat':r'.*aws13.*ihep.ac.cn','pvalue':8}]

In [8]:
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

indices = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')

indices = sorted(indices)
indices = [x for x in indices if x != '' and not x.endswith('_reindexed')]
#print(indices)
def time_filter(indices, last_days=1):
    if last_days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'jobs_archive_', '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

ind = time_filter(indices, 0)
print ("number of indices:", len(ind))
#print(ind)
ind = ','.join(ind)
#print(ind)

number of indices: 1


In [9]:
etime=int(time.time()*1000)
stime=etime-3600*24*last_days*1000
my_query={
  "size": 0,
  "aggs": {
    "task": {
      "terms": {
        "field": "jeditaskid",
        "size": 100,
        "order": {
          "_count": "desc"
        }
      }
    }
  },
  "query": {
    "bool": {
      "must": [
        {
          "match_all": {}
        },
        {
          "match_phrase": {
            "computingsite": {
              "query": "BOINC_MCORE"
            }
          }
        },
        {
          "match_phrase": {
            "jobstatus": {
              "query": "finished"
            }
          }
        },
        {
          "range": {
            "modificationtime": {
              "gte": stime,
              "lte": etime,
              "format": "epoch_millis"
            }
          }
        }
      ],
      "must_not": []
    }
   }
}
res = es.search(index=ind, body=my_query, request_timeout=12000)
#print(res)
buckets=res['aggregations']['task']['buckets']
tasks=[]
bs=len(buckets)
for i in range(bs):
    tid=buckets[i]['key']
    cn=buckets[i]['doc_count']
    #print(tid,cn)
    if cn>1000:
        tasks.append(tid)
print(tasks)

[13634372, 12515739, 12678480, 13183578, 12967708, 13092403, 13466969, 13293809, 12906983, 13240894, 12640557, 13417234, 13606525, 12790254, 12577096, 12732603, 13376330, 12858266, 12763289, 13346436, 13346443, 12885762, 12866455]


### now we want to find all the jobs of this taskid

In [10]:
def get_avg_boinc_power_by_task(tasks,res):
    (total_cpu,total_events,cn_boinc,ihep_total_cpu,ihep_total_events,cn_ihep)=({},{},{},{},{},{})
    for r in res:
        if r['_source']['cpuconsumptiontime']<1000000 and r['_source']['nevents']!=0 and "modificationhost" in r['_source'] and type(r['_source']['modificationhost']) is str:
            cpuperevent=float(r['_source']['cpuconsumptiontime'])/r['_source']['nevents']
        else:
            continue
        mhost=r['_source']['modificationhost']
        jediid=r['_source']['jeditaskid']
        if jediid in tasks:  
            if jediid not in total_cpu:
                total_cpu[jediid]=0
                total_events[jediid]=0
                cn_boinc[jediid]=0
                ihep_total_cpu[jediid]={}
                ihep_total_events[jediid]={}
                cn_ihep[jediid]={}
            else:
                total_cpu[jediid]+=r['_source']['cpuconsumptiontime']
                total_events[jediid]+=r['_source']['nevents']
                cn_boinc[jediid]+=1
            for P in Power:
                pat=P['pat']
                pvalue=P['pvalue']
                if re.match(pat,mhost):
                    if not pat in ihep_total_cpu[jediid]:
                        ihep_total_cpu[jediid][pat]=0
                        ihep_total_events[jediid][pat]=0
                        cn_ihep[jediid][pat]=0
                    else:
                        ihep_total_cpu[jediid][pat]+=r['_source']['cpuconsumptiontime']
                        ihep_total_events[jediid][pat]+=r['_source']['nevents']
                        cn_ihep[jediid][pat]+=1
    avg_cpuperevent={}
    for jediid in tasks:
        if total_events[jediid]!=0:
            avg_cpuperevent[jediid]=float(total_cpu[jediid])/total_events[jediid]
    ihep_avg_cpuperevent={}
    for jediid in ihep_total_cpu.keys():
        ihep_avg_cpuperevent[jediid]={}
        for pat in ihep_total_cpu[jediid].keys():
            if ihep_total_events[jediid][pat]!=0:
                ihep_avg_cpuperevent[jediid][pat]=float(ihep_total_cpu[jediid][pat])/ihep_total_events[jediid][pat]
            else:
                ihep_avg_cpuperevent[jediid][pat]=0
    
    head=["days","taskid","jobs","core power","avg_cpuperevent","nevents"]
    total_power_task=0
    total_cpu_task=0
    dlist=[]
   
    for jediid in ihep_total_cpu.keys():
        total_cpu_task+=total_cpu[jediid]
        boinc_power={}
        total_power,cn=0,0
        for pat in ihep_total_cpu[jediid]:
            if ihep_avg_cpuperevent[jediid][pat]!=0:
                for P in Power:
                    if pat in P.values():
                        boinc_power[jediid]=P["pvalue"]*ihep_avg_cpuperevent[jediid][pat]/avg_cpuperevent[jediid]
                    #print(pat,boinc_power)
                        total_power+=boinc_power[jediid]
                        cn+=1
        if cn!=0:
            avg_boinc_power=float(total_power)/cn
            total_power_task+=avg_boinc_power*total_cpu[jediid]
            #print("%d days\ttaskid:%s\tjobs:%s\tBOINC power:%.1f\tavg_cpuperevent:%.0f"%(last_days,jediid,cn_boinc[jediid],avg_boinc_power,avg_cpuperevent[jediid]))
            #print("%d\t%s\t%s\t%.1f\t%.0f"%(last_days,jediid,cn_boinc[jediid],avg_boinc_power,avg_cpuperevent[jediid]))
            t=[last_days,jediid,cn_boinc[jediid],avg_boinc_power,avg_cpuperevent[jediid],total_events[jediid]]
            dlist.append(t)
    avg_power=total_power_task/total_cpu_task
    print("Avg. power over all task:%.2f"%avg_power)
    df = pd.DataFrame(dlist,columns=head)
    print(df.sort_values(by="jobs",ascending=False))
    df.to_csv("plots/task.csv")
 
my_query={
    "size": 0,
    "_source": ["modificationhost", "cpuconsumptiontime","nevents","jeditaskid"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"jobstatus": "finished"},
          "term": {"computingsite":"BOINC_MCORE"}
        }
      ]
    }
  }
}
#for task in tasks:
res = helpers.scan(es, query=my_query, index=ind, scroll='5m', timeout="5m", size=1000)
get_avg_boinc_power_by_task(tasks,res)


Avg. power over all task:11.27
    days    taskid    jobs  core power  avg_cpuperevent   nevents
10   150  12515739  187476   12.534734       120.570525   9373800
1    150  13634372  138827   11.412876       175.071589  27765400
5    150  12678480  124478    9.522221       266.449059   6223900
9    150  13183578  122858   11.056078       317.599148   6142900
11   150  12967708  107730   11.527776       174.770192  21546000
18   150  13092403   96606   11.048342       476.484949   4830300
8    150  13466969   94353   11.572271       334.558557  18870600
16   150  13293809   92732   10.636647       214.867558  18546400
13   150  12906983   78368   11.360464       297.486257  15673500
22   150  13240894   75043   10.949287       346.196728   3752150
14   150  12640557   71670    9.706369       272.601105   3583500
6    150  13417234   61964   11.634448       211.160712  12392800
15   150  12790254   54049   12.018166       170.849819  10809800
21   150  13606525   49511   11.286888       